# Gráficos de Cleveland: Diferencias CT por Tipo de Ítem

Este notebook genera **gráficos de Cleveland comparativos** separando los ítems por tipo:
- **Panel A**: Ítems Progresistas
- **Panel B**: Ítems Conservadores

## Objetivo:

Visualizar si los patrones de cambio entre Generales y Ballotage difieren según el tipo de ítem.

## Interpretación:

- **Valores positivos**: Mayor tiempo en candidatos de Izquierda
- **Valores negativos**: Mayor tiempo en candidatos de Derecha
- **Punto azul**: Diferencia en Generales
- **Punto rojo**: Diferencia en Ballotage
- **Línea verde**: Aumentó en Ballotage
- **Línea naranja**: Disminuyó en Ballotage

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
from matplotlib.lines import Line2D
import warnings
warnings.filterwarnings('ignore')

# Configurar estilo
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette('husl')

print('✓ Librerías cargadas exitosamente')

## 1. Cargar Datos

In [ ]:
# Ruta al archivo
Ruta_Base = os.path.join(os.getcwd(), '..', 'Data', 'Procesados')
Archivo_Elecciones = os.path.join(Ruta_Base, 'df_Elecciones.xlsx')

# Cargar datos
df_Elecciones = pd.read_excel(Archivo_Elecciones)

print(f'✓ Datos cargados:')
print(f'  - {len(df_Elecciones)} registros')
print(f'  - {len(df_Elecciones.columns)} columnas')

# Verificar columnas
columnas_dif_gen = [col for col in df_Elecciones.columns if col.startswith('Dif_Gen_CT')]
columnas_dif_bal = [col for col in df_Elecciones.columns if col.startswith('Dif_Bal_CT')]

print(f'\n  - Variables Dif_Gen_CT: {len(columnas_dif_gen)}')
print(f'  - Variables Dif_Bal_CT: {len(columnas_dif_bal)}')

## 2. Preparar Datos para Gráfico

In [ ]:
# Definir ítems
Items_Progresistas = [5, 6, 9, 11, 16, 20, 24, 25, 27, 28]
Items_Conservadores = [3, 4, 7, 8, 10, 19, 22, 23, 29, 30]
Todos_Items = sorted(Items_Progresistas + Items_Conservadores)

print(f'Total de ítems: {len(Todos_Items)}')
print(f'Ítems Progresistas: {Items_Progresistas}')
print(f'Ítems Conservadores: {Items_Conservadores}')

# Calcular promedios por ítem
datos_grafico = []

for item in Todos_Items:
    var_gen = f'Dif_Gen_CT_Item_{item}'
    var_bal = f'Dif_Bal_CT_Item_{item}'
    
    if var_gen in df_Elecciones.columns and var_bal in df_Elecciones.columns:
        # Calcular promedios (ignorando NaN)
        media_gen = df_Elecciones[var_gen].mean()
        media_bal = df_Elecciones[var_bal].mean()
        
        # Calcular cambio
        cambio = media_bal - media_gen
        
        # Determinar tipo de ítem
        tipo = 'Progresista' if item in Items_Progresistas else 'Conservador'
        
        datos_grafico.append({
            'Item': item,
            'Tipo': tipo,
            'Dif_Generales': media_gen,
            'Dif_Ballotage': media_bal,
            'Cambio': cambio
        })

# Crear DataFrame
df_grafico = pd.DataFrame(datos_grafico)

# Separar por tipo
df_prog = df_grafico[df_grafico['Tipo'] == 'Progresista'].reset_index(drop=True)
df_cons = df_grafico[df_grafico['Tipo'] == 'Conservador'].reset_index(drop=True)

print(f'\n✓ Datos preparados:')
print(f'  - Ítems Progresistas: {len(df_prog)}')
print(f'  - Ítems Conservadores: {len(df_cons)}')
print(f'  - Total: {len(df_grafico)}')

## 3. Función para Crear Gráfico de Cleveland (Panel Individual)

In [ ]:
def Crear_Panel_Cleveland(df, ax, titulo, mostrar_ylabel=True):
    """
    Crea un panel de Cleveland en un eje específico.
    
    Parámetros:
    -----------
    df : DataFrame
        DataFrame con columnas: Item, Dif_Generales, Dif_Ballotage, Cambio
    ax : matplotlib.axes.Axes
        Eje donde dibujar
    titulo : str
        Título del panel
    mostrar_ylabel : bool
        Si mostrar etiqueta del eje Y
    """
    
    # Ordenar por cambio (descendente)
    df_sorted = df.sort_values('Cambio', ascending=True).reset_index(drop=True)
    
    # Parámetros visuales
    y_positions = np.arange(len(df_sorted))
    
    # Dibujar líneas conectando los puntos
    for idx, row in df_sorted.iterrows():
        gen_val = row['Dif_Generales']
        bal_val = row['Dif_Ballotage']
        cambio = row['Cambio']
        
        # Determinar color de línea según cambio
        if cambio > 0.5:  # Aumentó
            color_linea = '#2ecc71'  # Verde
            alpha = 0.6
        elif cambio < -0.5:  # Disminuyó
            color_linea = '#e74c3c'  # Rojo/Naranja
            alpha = 0.6
        else:  # Cambio pequeño
            color_linea = '#95a5a6'  # Gris
            alpha = 0.3
        
        # Dibujar línea
        ax.plot([gen_val, bal_val], [idx, idx], 
                color=color_linea, linewidth=1.5, alpha=alpha, zorder=1)
    
    # Dibujar puntos de Generales
    ax.scatter(df_sorted['Dif_Generales'], y_positions, 
               s=120, c='#3498db', marker='o', 
               edgecolors='white', linewidths=1.5,
               label='Generales', zorder=3, alpha=0.8)
    
    # Dibujar puntos de Ballotage
    ax.scatter(df_sorted['Dif_Ballotage'], y_positions, 
               s=120, c='#e74c3c', marker='o', 
               edgecolors='white', linewidths=1.5,
               label='Ballotage', zorder=3, alpha=0.8)
    
    # Línea vertical en x=0
    ax.axvline(x=0, color='black', linestyle='--', linewidth=0.8, alpha=0.3, zorder=0)
    
    # Configurar ejes
    ax.set_yticks(y_positions)
    ax.set_yticklabels([f'Ítem {int(item)}' for item in df_sorted['Item']], fontsize=9)
    
    ax.set_xlabel('Diferencia Izq - Der (segundos)', fontsize=11, fontweight='bold')
    if mostrar_ylabel:
        ax.set_ylabel('Ítem', fontsize=11, fontweight='bold')
    
    ax.set_title(titulo, fontsize=12, fontweight='bold', pad=15)
    
    # Grid
    ax.grid(True, axis='x', alpha=0.3, linestyle=':', linewidth=0.5)
    ax.set_axisbelow(True)
    
    # Ajustar límites del eje X para que ambos paneles tengan la misma escala
    return ax

## 4. Gráfico Comparativo: Progresistas vs Conservadores

In [ ]:
# Crear figura con 2 paneles lado a lado
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(20, 10), sharey=False)

# Determinar límites comunes del eje X
min_val = min(df_grafico['Dif_Generales'].min(), df_grafico['Dif_Ballotage'].min())
max_val = max(df_grafico['Dif_Generales'].max(), df_grafico['Dif_Ballotage'].max())
margen = (max_val - min_val) * 0.1

# Panel A: Ítems Progresistas
Crear_Panel_Cleveland(df_prog, ax1, 'A) Ítems Progresistas', mostrar_ylabel=True)
ax1.set_xlim(min_val - margen, max_val + margen)

# Panel B: Ítems Conservadores
Crear_Panel_Cleveland(df_cons, ax2, 'B) Ítems Conservadores', mostrar_ylabel=False)
ax2.set_xlim(min_val - margen, max_val + margen)

# Leyenda común en la parte superior
legend_elements = [
    Line2D([0], [0], marker='o', color='w', markerfacecolor='#3498db', 
           markersize=10, label='Generales', markeredgecolor='white', markeredgewidth=1),
    Line2D([0], [0], marker='o', color='w', markerfacecolor='#e74c3c', 
           markersize=10, label='Ballotage', markeredgecolor='white', markeredgewidth=1),
    Line2D([0], [0], color='#2ecc71', linewidth=2, label='Aumentó (>0.5s)'),
    Line2D([0], [0], color='#95a5a6', linewidth=2, label='Sin cambio (±0.5s)'),
    Line2D([0], [0], color='#e74c3c', linewidth=2, label='Disminuyó (<-0.5s)')
]

fig.legend(handles=legend_elements, loc='upper center', ncol=5, 
           fontsize=11, framealpha=0.95, edgecolor='gray', 
           bbox_to_anchor=(0.5, 0.98))

# Título general
fig.suptitle('Diferencias en Tiempo (Izq - Der): Generales vs Ballotage\nComparación por Tipo de Ítem',
             fontsize=14, fontweight='bold', y=0.995)

# Ajustar layout
plt.tight_layout(rect=[0, 0, 1, 0.96])

# Guardar
Carpeta_Destino = 'Graficos_Cleveland'
if not os.path.exists(Carpeta_Destino):
    os.makedirs(Carpeta_Destino)

ruta_completa = os.path.join(Carpeta_Destino, 'Cleveland_CT_Por_Tipo.png')
plt.savefig(ruta_completa, dpi=300, bbox_inches='tight', facecolor='white')
print(f'✅ Gráfico guardado: {ruta_completa}')

plt.show()

## 4.1. Análisis de Significancia Estadística

In [ ]:
print('='*70)
print('ANÁLISIS DE SIGNIFICANCIA ESTADÍSTICA')
print('='*70)

for tipo, df_tipo in [('Progresistas', df_prog), ('Conservadores', df_cons)]:
    print(f'\n📊 Ítems {tipo}:')
    print('-'*70)
    
    sig_items = df_tipo[df_tipo['Significancia'] != '']
    print(f'  Total de ítems: {len(df_tipo)}')
    print(f'  Ítems significativos: {len(sig_items)} ({len(sig_items)/len(df_tipo)*100:.1f}%)')
    
    if len(sig_items) > 0:
        n_p001 = len(df_tipo[df_tipo['Significancia'] == '***'])
        n_p01 = len(df_tipo[df_tipo['Significancia'] == '**'])
        n_p05 = len(df_tipo[df_tipo['Significancia'] == '*'])
        
        print(f'\n  Desglose:')
        print(f'    *** (p < 0.001): {n_p001} ítems')
        print(f'    **  (p < 0.01):  {n_p01} ítems')
        print(f'    *   (p < 0.05):  {n_p05} ítems')
        
        print(f'\n  Ítems significativos:')
        sig_sorted = sig_items.sort_values('p_valor')[['Item', 'Cambio', 'p_valor', 'Significancia']]
        for idx, row in sig_sorted.iterrows():
            print(f"    Ítem {int(row['Item']):2d}: Cambio = {row['Cambio']:6.3f}s, p = {row['p_valor']:.4f} {row['Significancia']}")
    else:
        print('  ⚠️  Ningún ítem significativo')

print('\n' + '='*70)

## 5. Análisis Comparativo por Tipo

In [ ]:
print('='*70)
print('ANÁLISIS COMPARATIVO: PROGRESISTAS vs CONSERVADORES')
print('='*70)

print('\n📊 ÍTEMS PROGRESISTAS:')
print('-'*70)
print(f'  Cantidad: {len(df_prog)} ítems')
print(f'  Promedio Dif_Generales: {df_prog["Dif_Generales"].mean():.3f} seg')
print(f'  Promedio Dif_Ballotage: {df_prog["Dif_Ballotage"].mean():.3f} seg')
print(f'  Promedio Cambio: {df_prog["Cambio"].mean():.3f} seg')

aumentaron_prog = len(df_prog[df_prog['Cambio'] > 0.5])
sin_cambio_prog = len(df_prog[(df_prog['Cambio'] >= -0.5) & (df_prog['Cambio'] <= 0.5)])
disminuyeron_prog = len(df_prog[df_prog['Cambio'] < -0.5])

print(f'\n  Distribución de cambios:')
print(f'    - Aumentaron: {aumentaron_prog} ({aumentaron_prog/len(df_prog)*100:.1f}%)')
print(f'    - Sin cambio: {sin_cambio_prog} ({sin_cambio_prog/len(df_prog)*100:.1f}%)')
print(f'    - Disminuyeron: {disminuyeron_prog} ({disminuyeron_prog/len(df_prog)*100:.1f}%)')

print('\n📊 ÍTEMS CONSERVADORES:')
print('-'*70)
print(f'  Cantidad: {len(df_cons)} ítems')
print(f'  Promedio Dif_Generales: {df_cons["Dif_Generales"].mean():.3f} seg')
print(f'  Promedio Dif_Ballotage: {df_cons["Dif_Ballotage"].mean():.3f} seg')
print(f'  Promedio Cambio: {df_cons["Cambio"].mean():.3f} seg')

aumentaron_cons = len(df_cons[df_cons['Cambio'] > 0.5])
sin_cambio_cons = len(df_cons[(df_cons['Cambio'] >= -0.5) & (df_cons['Cambio'] <= 0.5)])
disminuyeron_cons = len(df_cons[df_cons['Cambio'] < -0.5])

print(f'\n  Distribución de cambios:')
print(f'    - Aumentaron: {aumentaron_cons} ({aumentaron_cons/len(df_cons)*100:.1f}%)')
print(f'    - Sin cambio: {sin_cambio_cons} ({sin_cambio_cons/len(df_cons)*100:.1f}%)')
print(f'    - Disminuyeron: {disminuyeron_cons} ({disminuyeron_cons/len(df_cons)*100:.1f}%)')

print('\n' + '='*70)

## 6. Comparación Estadística

In [ ]:
from scipy import stats

print('='*70)
print('COMPARACIÓN ESTADÍSTICA')
print('='*70)

# Test t para comparar cambios entre tipos
t_stat, p_value = stats.ttest_ind(df_prog['Cambio'], df_cons['Cambio'])

print('\n📊 Test t de Student: Cambio Promedio')
print('-'*70)
print(f'  H0: Los cambios promedio son iguales entre tipos')
print(f'  Ha: Los cambios promedio son diferentes entre tipos')
print(f'\n  Estadístico t: {t_stat:.4f}')
print(f'  Valor p: {p_value:.4f}')

if p_value < 0.05:
    print(f'\n  ✅ SIGNIFICATIVO (p < 0.05)')
    print(f'     Los ítems {"Progresistas" if df_prog["Cambio"].mean() > df_cons["Cambio"].mean() else "Conservadores"} muestran mayor cambio promedio')
else:
    print(f'\n  ❌ NO SIGNIFICATIVO (p ≥ 0.05)')
    print(f'     No hay diferencia significativa en el cambio promedio entre tipos')

# Diferencia de medias
dif_medias = df_prog['Cambio'].mean() - df_cons['Cambio'].mean()
print(f'\n  Diferencia de medias: {dif_medias:.4f} seg')
print(f'  (Progresistas - Conservadores)')

print('\n' + '='*70)

## 7. Top Ítems por Tipo

In [ ]:
print('='*70)
print('TOP ÍTEMS CON MAYOR CAMBIO (POR TIPO)')
print('='*70)

print('\n🏆 ÍTEMS PROGRESISTAS - Mayor Aumento:')
print('-'*70)
top_aumento_prog = df_prog.nlargest(5, 'Cambio')[['Item', 'Dif_Generales', 'Dif_Ballotage', 'Cambio']]
print(top_aumento_prog.to_string(index=False))

print('\n🏆 ÍTEMS PROGRESISTAS - Mayor Disminución:')
print('-'*70)
top_disminucion_prog = df_prog.nsmallest(5, 'Cambio')[['Item', 'Dif_Generales', 'Dif_Ballotage', 'Cambio']]
print(top_disminucion_prog.to_string(index=False))

print('\n🏆 ÍTEMS CONSERVADORES - Mayor Aumento:')
print('-'*70)
top_aumento_cons = df_cons.nlargest(5, 'Cambio')[['Item', 'Dif_Generales', 'Dif_Ballotage', 'Cambio']]
print(top_aumento_cons.to_string(index=False))

print('\n🏆 ÍTEMS CONSERVADORES - Mayor Disminución:')
print('-'*70)
top_disminucion_cons = df_cons.nsmallest(5, 'Cambio')[['Item', 'Dif_Generales', 'Dif_Ballotage', 'Cambio']]
print(top_disminucion_cons.to_string(index=False))

print('\n' + '='*70)

## 8. Guardar Tablas Resumen

In [ ]:
# Crear carpeta de salida
Carpeta_Salida = os.path.join(os.getcwd(), '..', 'Data', 'Resultados_Cleveland')
if not os.path.exists(Carpeta_Salida):
    os.makedirs(Carpeta_Salida)

# Guardar tablas separadas por tipo
Archivo_Prog = os.path.join(Carpeta_Salida, 'Resumen_CT_Progresistas.xlsx')
Archivo_Cons = os.path.join(Carpeta_Salida, 'Resumen_CT_Conservadores.xlsx')

df_prog.to_excel(Archivo_Prog, index=False)
df_cons.to_excel(Archivo_Cons, index=False)

print(f'✅ Tablas guardadas:')
print(f'   - {Archivo_Prog}')
print(f'   - {Archivo_Cons}')

# Crear tabla comparativa
comparacion = pd.DataFrame({
    'Tipo': ['Progresistas', 'Conservadores'],
    'N_Items': [len(df_prog), len(df_cons)],
    'Media_Dif_Gen': [df_prog['Dif_Generales'].mean(), df_cons['Dif_Generales'].mean()],
    'Media_Dif_Bal': [df_prog['Dif_Ballotage'].mean(), df_cons['Dif_Ballotage'].mean()],
    'Media_Cambio': [df_prog['Cambio'].mean(), df_cons['Cambio'].mean()],
    'Aumentaron_N': [aumentaron_prog, aumentaron_cons],
    'Sin_Cambio_N': [sin_cambio_prog, sin_cambio_cons],
    'Disminuyeron_N': [disminuyeron_prog, disminuyeron_cons]
})

Archivo_Comparacion = os.path.join(Carpeta_Salida, 'Comparacion_Por_Tipo.xlsx')
comparacion.to_excel(Archivo_Comparacion, index=False)

print(f'   - {Archivo_Comparacion}')
print(f'\n✓ Total: 3 archivos Excel generados')

## 9. Resumen Final

In [ ]:
print('='*70)
print('RESUMEN: GRÁFICOS DE CLEVELAND POR TIPO DE ÍTEM')
print('='*70)

print('\n📊 Análisis completado:')
print(f'  - Ítems Progresistas: {len(df_prog)}')
print(f'  - Ítems Conservadores: {len(df_cons)}')
print(f'  - Gráficos generados: 1 (2 paneles comparativos)')

print('\n📁 Archivos generados:')
print('  - Cleveland_CT_Por_Tipo.png (gráfico comparativo)')
print('  - Resumen_CT_Progresistas.xlsx')
print('  - Resumen_CT_Conservadores.xlsx')
print('  - Comparacion_Por_Tipo.xlsx')

print('\n🎯 Hallazgos clave:')
print(f'\n  PROGRESISTAS:')
print(f'    - Cambio promedio: {df_prog["Cambio"].mean():.3f} seg')
print(f'    - Aumentaron: {aumentaron_prog}/{len(df_prog)} ítems')
print(f'    - Disminuyeron: {disminuyeron_prog}/{len(df_prog)} ítems')

print(f'\n  CONSERVADORES:')
print(f'    - Cambio promedio: {df_cons["Cambio"].mean():.3f} seg')
print(f'    - Aumentaron: {aumentaron_cons}/{len(df_cons)} ítems')
print(f'    - Disminuyeron: {disminuyeron_cons}/{len(df_cons)} ítems')

print('\n💡 Interpretación:')
if abs(dif_medias) < 0.1:
    print('  ✓ Ambos tipos de ítems muestran patrones similares de cambio')
elif dif_medias > 0:
    print('  ✓ Los ítems Progresistas muestran MAYOR incremento en asimetría Izq-Der')
else:
    print('  ✓ Los ítems Conservadores muestran MAYOR incremento en asimetría Izq-Der')

print('\n' + '='*70)
print('✅ ANÁLISIS COMPLETADO')
print('='*70)